<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

In [1]:
# Parameters
EMO_INDEX = 2
MODEL_INDEX = 3
N = 20


In [2]:
import os
import ast
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.dates as mdates
import statsmodels.api as sm # 用于统计检验
%matplotlib inline
plt.rcParams["font.sans-serif"]=["WenQuanYi Micro Hei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

## 参数选择

In [3]:
# # Parameters
# CLUSTER_INDEX = 2
# EMO_INDEX = 0

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
ORI_DATA_PATH = '/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Temporal_Clustering/data'
ROOT_PATH = '/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Result_Analysis'
CLUSTER_PATH = f'{ROOT_PATH}/data/Comparison_of_Clustering_methods'

CLUSTER = ['Density', 'Features', 'Representation', 'Shape'][CLUSTER_INDEX]
Emotion_Data_PATH = f'{ORI_DATA_PATH}/Emotion_Data/Deep-learning/BERT'   # 情绪数据路径
Financial_Data_PATH = f'{ORI_DATA_PATH}/Financial_Data' # 金融数据路径

NameError: name 'CLUSTER_INDEX' is not defined

In [ ]:
EMO_MARKET = ['Shanghai_Composite_Index', 'CSI_300_Index', 'Chinext_Index'][EMO_INDEX]
EMO_INDEX_MAP = {
    'Shanghai_Composite_Index': '上证综合情绪值',
    'CSI_300_Index': '沪深300情绪值',
    'Chinext_Index': '创业板情绪值'
}
EMO_NAME = EMO_INDEX_MAP[EMO_MARKET]

## 数据预处理

In [ ]:
## 读取聚类结果
cluster_data = pd.read_csv(f'{CLUSTER_PATH}/{EMO_MARKET}/{CLUSTER}.csv', dtype={"Stkcd": str})
cluster_data.columns = ['股票编号', '聚类标签', '公司名称']
cluster_data.head()

In [ ]:
## 读取股吧个股的数据
all_data = []
file_list = [f for f in os.listdir(Emotion_Data_PATH) if f.endswith('.csv')]

for file in file_list:
    file_path = os.path.join(Emotion_Data_PATH, file)
    df = pd.read_csv(file_path)
    stock_code = os.path.splitext(file)[0] # 获取股票编号（文件名去掉扩展名）
    
    # 提取每一行的日期和情绪值
    for _, row in df.iterrows():
        new_row = {
            '股票编号': stock_code,
            '日期': row['日期'],
            '上证综合情绪值': row['上证综合情绪值'],
            '沪深300情绪值': row['沪深300情绪值'],
            '创业板情绪值': row['创业板情绪值']
        }
        all_data.append(new_row)
guba_data = pd.DataFrame(all_data)
guba_data

In [ ]:
## 读取股票回报率的数据
return_data = pd.read_csv(f'{Financial_Data_PATH}/日个股回报率.csv', dtype={'股票编号': str})
return_data

In [ ]:
## 进行左连接，以 guba_data 为主表
merged_data = pd.merge(guba_data, return_data[['股票编号', '日期', '交易量', '收益率变化']], 
                       on=['股票编号', '日期'], 
                       how='left')
merged_data = pd.merge(merged_data, cluster_data, on='股票编号', how='left')
merged_data = merged_data.dropna()

merged_data['日期'] = pd.to_datetime(merged_data['日期'])
merged_data = merged_data[(merged_data['日期'] > '2021-05-06') & (merged_data['日期'] < '2024-11-18')]
merged_data

In [ ]:
# 对情绪值列进行 Min-Max 标准化
def min_max_normalization(df, cols):
    for col in cols:
        min_val = df[col].min()
        max_val = df[col].max()
        df[col] = 2 * (df[col] - min_val) / (max_val - min_val) - 1
    return df

# 对每个股票编号的数据进行标准化和按日期汇总
def process_data(df):
    df_processed = pd.DataFrame()  # 用于存储结果
    for stock_code, stock_data in df.groupby('股票编号'):
        # 对每个股票编号内的数据进行标准化
        stock_data = min_max_normalization(stock_data, ['上证综合情绪值', '沪深300情绪值', '创业板情绪值'])
        
        # 按日期汇总数据，同时保留股票编号
        stock_summary = stock_data.groupby('日期').agg({
            '股票编号': 'first',  # 保留股票编号（在同一日期内它是相同的，使用 'first'）
            '上证综合情绪值': 'mean',  # 上证综合情绪值按日期取均值
            '沪深300情绪值': 'mean',  # 沪深300情绪值按日期取均值
            '创业板情绪值': 'mean',  # 创业板情绪值按日期取均值
            '交易量': 'mean',  # 交易量按日期求和
            '收益率变化': 'mean',  # 收益率变化按日期取均值
            '聚类标签': 'first', # 保留聚类标签（在同一日期内它是相同的，使用 'first'）
            '公司名称': 'first', # 保留公司名称（在同一日期内它是相同的，使用 'first'）
        }).reset_index(drop=False)
        
        # 合并每个股票的汇总数据
        df_processed = pd.concat([df_processed, stock_summary], ignore_index=True)
        df_processed = df_processed.sort_values(by=['股票编号', '日期'], ascending=[True, True])
    return df_processed

final_data = process_data(merged_data)
final_data

In [ ]:
# 统计检验 : 按照聚类标签分组，并计算均值和方差
df = final_data.copy()
statistical_result = df.groupby('聚类标签').agg(
    情绪值均值=(EMO_NAME, 'mean'),
    情绪值方差=(EMO_NAME, 'var'),
    收益率变化均值=('收益率变化', 'mean'),
    收益率变化方差=('收益率变化', 'var')
).reset_index()

# 输出结果
statistical_result

## 情绪值与股价二维展示图

In [ ]:
## 绘制不同簇情绪值随时间变化图

# 按 '聚类标签' 和 '日期' 分组，计算平均情绪值
df_grouped = final_data.groupby(['聚类标签', '日期'])[['上证综合情绪值', '沪深300情绪值', '创业板情绪值']].mean().reset_index()

# 为每个聚类标签绘制折线
i = 0
plt.figure(figsize=(12, 12))
for cluster in df_grouped['聚类标签'].unique():
    if i == 0:
        x = 'red'
    elif i == 1:
        x = 'blue'
    elif i == 2:
        x = 'green'
    df_cluster = df_grouped[df_grouped['聚类标签'] == cluster]
    plt.plot(df_cluster['日期'], df_cluster[EMO_NAME], label=f'{EMO_NAME} - 聚类 {cluster}', color=x)
    i += 1
    
plt.xlabel('日期')
plt.ylabel('平均情绪值')
plt.title('情绪值趋势图')


plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=3))  # 每三个月显示
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 格式化为年-月

plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()

# plt.savefig(f'个股分析/{market_address}/图片/情绪值趋势图', dpi=300)
plt.show()


In [ ]:
## 绘制不同簇收益率随时间变化图

# 按 '聚类标签' 和 '日期' 分组，计算平均收益率
return_grouped = final_data.groupby(['聚类标签', '日期'])['收益率变化'].mean().reset_index()

# 为每个聚类标签绘制折线
i = 0
plt.figure(figsize=(12, 12))
for cluster in df_grouped['聚类标签'].unique():
    if i == 0:
        x = 'red'
    elif i == 1:
        x = 'blue'
    elif i == 2:
        x = 'green'
    df_cluster = return_grouped[return_grouped['聚类标签'] == cluster]
    plt.plot(df_cluster['日期'], df_cluster['收益率变化'], label=f'收益率变化 - 聚类 {cluster}', color=x)
    i += 1

plt.xlabel('日期')
plt.ylabel('平均收益率')
plt.title('收益率趋势图')

plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=3))  # 每三个月显示
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 格式化为年-月

plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()

# 显示图表
# plt.savefig(f'个股分析/{market_address}/图片/收益率趋势图', dpi=300)
plt.show()

## 三维展示图，分别展示簇是0，1，2的代表图

In [ ]:
### 聚类0示例 ###

# 复制数据集，确保不会修改原始数据
merged_data_cleaned = final_data.copy()
merged_data_cleaned['日期'] = pd.to_datetime(merged_data_cleaned['日期'])

# 选择聚类标签为 0 的数据，找到数据最多的股票编号
cluster_0_data = merged_data_cleaned[merged_data_cleaned['聚类标签'] == 0]
top_stock_code = cluster_0_data['股票编号'].value_counts().idxmax()
top_stock_data = cluster_0_data[cluster_0_data['股票编号'] == top_stock_code]

# 创建 3D 图
fig = plt.figure(figsize=(50, 50))
ax = fig.add_subplot(111, projection='3d')

# 提取绘图数据
x = top_stock_data[EMO_NAME]  # 情绪值作为 x 轴
y = mdates.date2num(top_stock_data['日期'])  # 时间转换为数值类型
z = top_stock_data['收益率变化']  # 收益率变化作为 z 轴

# 绘制三维折线图
ax.plot(x, y, z, label=f"股票 {top_stock_code}", marker='o', linestyle='-', alpha=0.7)

# 设置轴标签
ax.set_xlabel(EMO_NAME)
ax.set_ylabel('时间')
ax.set_zlabel('收益率变化')
ax.set_title(f'股票 {top_stock_code}: {EMO_NAME} 与收益率变化的三维关系')

# 设置 y 轴时间刻度（每半年一个刻度）
ax.yaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=6))  # 每半年一个刻度
ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 按年月显示

# 旋转日期标签防止重叠
fig.autofmt_xdate()

# 调整布局并保存图像
plt.legend()
# plt.savefig(f'个股分析/{market_address}/图片/聚类0_{top_stock_code}.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
### 聚类1示例 ###
# 复制数据集，确保不会修改原始数据
merged_data_cleaned = final_data.copy()
merged_data_cleaned['日期'] = pd.to_datetime(merged_data_cleaned['日期'])

# 选择聚类标签为 0 的数据，找到数据最多的股票编号
cluster_0_data = merged_data_cleaned[merged_data_cleaned['聚类标签'] == 1]
top_stock_code = cluster_0_data['股票编号'].value_counts().idxmax()
top_stock_data = cluster_0_data[cluster_0_data['股票编号'] == top_stock_code]


# 创建 3D 图
fig = plt.figure(figsize=(50, 50))
ax = fig.add_subplot(111, projection='3d')

# 提取绘图数据
x = top_stock_data[EMO_NAME]  # 情绪值作为 x 轴
y = mdates.date2num(top_stock_data['日期'])  # 时间转换为数值类型
z = top_stock_data['收益率变化']  # 收益率变化作为 z 轴

# 绘制三维折线图
ax.plot(x, y, z, label=f"股票 {top_stock_code}", marker='o', linestyle='-', alpha=0.7)

# 设置轴标签
ax.set_xlabel(EMO_NAME)
ax.set_ylabel('时间')
ax.set_zlabel('收益率变化')
ax.set_title(f'股票 {top_stock_code}: {EMO_NAME} 与收益率变化的三维关系')

# 设置 y 轴时间刻度（每半年一个刻度）
ax.yaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=6))  # 每半年一个刻度
ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 按年月显示

# 旋转日期标签防止重叠
fig.autofmt_xdate()

# 调整布局并保存图像
plt.legend()
# plt.savefig(f'个股分析/{market_address}/图片/聚类1_{top_stock_code}.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
### 聚类2示例 ###
# 复制数据集，确保不会修改原始数据
merged_data_cleaned = final_data.copy()
merged_data_cleaned['日期'] = pd.to_datetime(merged_data_cleaned['日期'])

# 选择聚类标签为 0 的数据，找到数据最多的股票编号
cluster_0_data = merged_data_cleaned[merged_data_cleaned['聚类标签'] == 2]
top_stock_code = cluster_0_data['股票编号'].value_counts().idxmax()
top_stock_data = cluster_0_data[cluster_0_data['股票编号'] == top_stock_code]


# 创建 3D 图
fig = plt.figure(figsize=(50, 50))
ax = fig.add_subplot(111, projection='3d')

# 提取绘图数据
x = top_stock_data[EMO_NAME]  # 情绪值作为 x 轴
y = mdates.date2num(top_stock_data['日期'])  # 时间转换为数值类型
z = top_stock_data['收益率变化']  # 收益率变化作为 z 轴

# 绘制三维折线图
ax.plot(x, y, z, label=f"股票 {top_stock_code}", marker='o', linestyle='-', alpha=0.7)

# 设置轴标签
ax.set_xlabel(EMO_NAME)
ax.set_ylabel('时间')
ax.set_zlabel('收益率变化')
ax.set_title(f'股票 {top_stock_code}: {EMO_NAME} 与收益率变化的三维关系')

# 设置 y 轴时间刻度（每半年一个刻度）
ax.yaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=6))  # 每半年一个刻度
ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 按年月显示

# 旋转日期标签防止重叠
fig.autofmt_xdate()

# 调整布局并保存图像
plt.legend()
# plt.savefig(f'个股分析/{market_address}/图片/聚类2_{top_stock_code}.png', dpi=300, bbox_inches='tight')
plt.show()


## 进行聚类间实证分析

#### 读取个股金融数据

In [ ]:
# 读取原始数据
microfinancial_data = pd.read_csv(f'{Financial_Data_PATH}/个股微观金融数据.csv', encoding='utf-8')

# 选择重要的列，日期和股票代码一定要包含
columns_to_select = [
    '日期_Date',  # 日期
    '股票代码_Stkcd',  # 股票代码
    '日振幅(%)_Dampltd',  # 日振幅
    '日收益率_Dret',  # 日收益率
    '市盈率_PE',  # 市盈率
    '成交量_Trdvol',  # 成交量
    '流通股日换手率(%)_DTrdTurnR',
    '总市值加权平均日资本收益_Daretmc',
]

# 提取相关列
microfinancial_data = microfinancial_data[columns_to_select]
microfinancial_data['股票代码_Stkcd'] = microfinancial_data['股票代码_Stkcd'].apply(lambda x: str(x).zfill(6))

# microfinancial_data = microfinancial_data[microfinancial_data['日期_Date'] == '2022-03-01'] #用于检验某一天数据
microfinancial_data

#### 加载之前的情绪聚类数据

In [ ]:
final_data

#### 读取市场换手率与市盈率并进行插值

In [ ]:
turnover_PE_rate = pd.read_csv(f'{Financial_Data_PATH}/市场整体换手率与市盈率.csv', encoding='utf-8')
turnover_PE_rate['统计日期'] = pd.to_datetime(turnover_PE_rate['统计日期'])

# 按照 '统计日期' 排序
turnover_PE_rate = turnover_PE_rate.sort_values(by='统计日期')

# 对 '平均市盈率' 和 '换手率(总股本)' 进行线性插值
turnover_PE_rate[['平均市盈率', '换手率(总股本)']] = turnover_PE_rate[['平均市盈率', '换手率(总股本)']].interpolate(method='linear', axis=0)

turnover_PE_rate = turnover_PE_rate.drop(columns=['统计口径编码'])
turnover_PE_rate.columns = ['统计日期', '市场平均市盈率', '市场换手率(总股本)']
turnover_PE_rate

#### 数据拼接

In [ ]:
# 将日期列转换为 datetime 格式
final_data['日期'] = pd.to_datetime(final_data['日期'])
microfinancial_data['日期_Date'] = pd.to_datetime(microfinancial_data['日期_Date'], errors='coerce')

# 使用 merge 进行内连接
result = pd.merge(
    final_data, 
    microfinancial_data, 
    left_on=['日期', '股票编号'],  # 左表的 key
    right_on=['日期_Date', '股票代码_Stkcd'], # 右表的 key
    how='inner'                   # 内连接
)

result = pd.merge(
    result, 
    turnover_PE_rate, 
    left_on=['日期'],  # 左表的 key
    right_on=['统计日期'], # 右表的 key
    how='inner'                   # 内连接
)

# 删除重复的列
result = result.drop(columns=['日期_Date', '股票代码_Stkcd', '交易量', '日收益率_Dret', '统计日期'])

# 修改列的位置，更加美观可视
columns_to_move = ['聚类标签', '公司名称']
result = pd.concat([result.drop(columns=columns_to_move), result[columns_to_move]], axis=1)

# 将成交量除以10000，并以万计数
result['成交量(百万)_Trdvol'] = result['成交量_Trdvol'] / 1_000_000
result = result.drop(columns=['成交量_Trdvol'])

cols_to_check = ['收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
                 '成交量(百万)_Trdvol', '流通股日换手率(%)_DTrdTurnR']

# 删除含有空值的行
result = result.dropna(subset=cols_to_check)
result

In [ ]:
# 查看匹配的公司数
unique_stock_count = result['股票编号'].nunique()
print(unique_stock_count)

In [ ]:
## 单因子：总市值加权平均日资本收益回归检验
df = result.copy()

# 自变量是总市值加权平均日资本收益
X = df[['总市值加权平均日资本收益_Daretmc']]

# 要拟合的因变量列表
y_vars = [
    '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
    '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
]


# 添加常数项（截距项）
X_with_const = sm.add_constant(X)

# 创建一个空的列表存储回归结果
regression_results = []

# 对每个因变量进行回归分析
for y_var in y_vars:
    y = df[y_var]  # 当前的因变量

    # 对股票编号分组进行回归
    grouped = df.groupby('股票编号')
    
    r_squared_list = []
    
    for stock_code, group in grouped:
        X_group = group[['总市值加权平均日资本收益_Daretmc']]
        X_with_const_group = sm.add_constant(X_group)
        y_group = group[y_var]
        
        try:
            # 拟合回归模型
            model = sm.OLS(y_group, X_with_const_group)
            results = model.fit()

            # 提取回归R方
            r_squared_list.append(results.rsquared)
        except Exception as e:
            # 如果回归失败（如R方计算为-inf），跳过
            print(f"回归失败：股票编号 {stock_code}, 错误信息: {e}")
            r_squared_list.append(float('nan'))  # 将其R方设为NaN
    
    # 计算R方的平均值，忽略NaN值
    avg_r_squared = pd.Series(r_squared_list).mean()
    
    # 对于每个因变量，存储回归系数等信息
    model = sm.OLS(y, X_with_const)
    results = model.fit()

    regression_results.append({
        'Variable': y_var,
        'S': round(results.bse['总市值加权平均日资本收益_Daretmc'], 3),  # 标准误差
        'Econ': round(results.params['总市值加权平均日资本收益_Daretmc'], 3),  # 回归系数
        'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
        'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
        'Econ_t_value': round(results.tvalues['总市值加权平均日资本收益_Daretmc'], 3),  # Econ的t统计量，保留三位小数
        'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
    })

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/单因子：总市值加权平均日资本收益.csv')
regression_df


In [ ]:
## 单因子：情绪回归检验

df = result.copy()

# 自变量是情绪值
X = df[[EMO_NAME]]

# 要拟合的因变量列表
y_vars = [
    '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
    '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
]

# 添加常数项（截距项）
X_with_const = sm.add_constant(X)

# 创建一个空的列表存储回归结果
regression_results = []

# 对每个因变量进行回归分析
for y_var in y_vars:
    y = df[y_var]  # 当前的因变量

    # 对股票编号分组进行回归
    grouped = df.groupby('股票编号')
    
    for stock_code, group in grouped:
        X_group = group[[EMO_NAME]]
        X_with_const_group = sm.add_constant(X_group)
        y_group = group[y_var]
        
        # 拟合回归模型
        model = sm.OLS(y_group, X_with_const_group)
        results = model.fit()

        # 提取回归R方
        r_squared_list.append(results.rsquared)
    
    # 计算R方的平均值
    avg_r_squared = sum(r_squared_list) / len(r_squared_list)
    
    # 对于每个因变量，存储回归系数等信息
    model = sm.OLS(y, X_with_const)
    results = model.fit()

    regression_results.append({
        'Variable': y_var,
        'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
        'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
        'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
        'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
        'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
        'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
    })

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/单因子：{EMO_NAME}.csv')
regression_df


In [ ]:
## 总体双因子回归检验
df = result.copy()

# 自变量是情绪值和总市值加权平均日资本收益
X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]

# 要拟合的因变量列表
y_vars = [
    '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
    '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
]

# 添加常数项（截距项）
X_with_const = sm.add_constant(X)

# 创建一个空的列表存储回归结果
regression_results = []

# 对每个因变量进行回归分析
for y_var in y_vars:
    y = df[y_var]  # 当前的因变量

    # 对股票编号分组进行回归
    grouped = df.groupby('股票编号')

    # 对于每个因变量，存储回归系数等信息
    model = sm.OLS(y, X_with_const)
    results = model.fit()

    regression_results.append({
        'Variable': y_var,
        'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
        'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
        'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
        'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
        'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
        'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
    })


# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：总体分析.csv')
regression_df


In [ ]:
## 聚类0的回归检验
df = result[result['聚类标签'] == 0].copy()

# 自变量是情绪值和总市值加权平均日资本收益
X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]

# 要拟合的因变量列表
y_vars = [
    '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
    '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
]


# 添加常数项（截距项）
X_with_const = sm.add_constant(X)

# 创建一个空的列表存储回归结果
regression_results = []

# 对每个因变量进行回归分析
for y_var in y_vars:
    y = df[y_var]  # 当前的因变量

    # 对股票编号分组进行回归
    grouped = df.groupby('股票编号')
    
    # 对于每个因变量，存储回归系数等信息
    model = sm.OLS(y, X_with_const)
    results = model.fit()

    regression_results.append({
        'Variable': y_var,
        'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
        'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
        'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
        'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
        'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
        'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
    })

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：聚类0检验.csv')
regression_df


In [ ]:
## 聚类1的回归检验
df = result[result['聚类标签'] == 1].copy()

# 自变量是情绪值和总市值加权平均日资本收益
X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]

# 要拟合的因变量列表
y_vars = [
    '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
    '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
]


# 添加常数项（截距项）
X_with_const = sm.add_constant(X)

# 创建一个空的列表存储回归结果
regression_results = []

# 对每个因变量进行回归分析
for y_var in y_vars:
    y = df[y_var]  # 当前的因变量

    # 对股票编号分组进行回归
    grouped = df.groupby('股票编号')

    # 对于每个因变量，存储回归系数等信息
    model = sm.OLS(y, X_with_const)
    results = model.fit()

    regression_results.append({
        'Variable': y_var,
        'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
        'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
        'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
        'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
        'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
        'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
    })


# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：聚类1检验.csv')
regression_df


In [ ]:
## 聚类2的回归检验
df = result[result['聚类标签'] == 2].copy()

# 自变量是情绪值和总市值加权平均日资本收益
X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]

# 要拟合的因变量列表
y_vars = [
    '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
    '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
]


# 添加常数项（截距项）
X_with_const = sm.add_constant(X)

# 创建一个空的列表存储回归结果
regression_results = []

# 对每个因变量进行回归分析
for y_var in y_vars:
    y = df[y_var]  # 当前的因变量

    # 对股票编号分组进行回归
    grouped = df.groupby('股票编号')

    # 对于每个因变量，存储回归系数等信息
    model = sm.OLS(y, X_with_const)
    results = model.fit()

    regression_results.append({
        'Variable': y_var,
        'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
        'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
        'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
        'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
        'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
        'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
    })

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：聚类2检验.csv')
regression_df
